In [4]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [6]:
!pip install -q transformers

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.6/7.6 MB 54.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 268.8/268.8 kB 25.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.8/7.8 MB 99.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 71.8 MB/s eta 0:00:00


In [7]:
from transformers import pipeline
import pandas as pd
import numpy as np

In [8]:
df = pd.read_excel('/content/drive/MyDrive/Tower_Sentiment.xlsx', 'TOWER REAL')
# Dropping columns that do not need to be read
df.drop(df.columns[[4, 5, 8, 9]], axis=1, inplace=True)

# Truncate the DataFrame to contain only the first 3007 rows
df = df.iloc[:3007]

#Current amount of rows in dataset
print(len(df))

#dropping rows after 1500 because thats where we annotated to
#df.drop(df.index[1500:], inplace=True)

#Dropping all NaN data points
df_filled = df.dropna()

#Reset the "index" of the dataset
df_filled = df_filled.reset_index(drop = True)

#df_filled = df.fillna(" ")
old_word = 'Neural'
new_word = 'Neutral'

# Find rows containing the target word and replace it with the new word
df_filled['Sentiment'] = df_filled['Sentiment'].replace(old_word, new_word)

old_word = 'Positve'
new_word = 'Positive'
# Find rows containing the target word and replace it with the new word
df_filled['Sentiment'] = df_filled['Sentiment'].replace(old_word, new_word)

old_word = 'positive'
new_word = 'Positive'
# Find rows containing the target word and replace it with the new word
df_filled['Sentiment'] = df_filled['Sentiment'].replace(old_word, new_word)

# Save the modified DataFrame back to the CSV file
#df_filled.to_csv('./Tower_Sentiment_New.xlsx', index=False)

#Removing all mistakes from the dataset where names were not spelt right
specific_word = 'Neural'
df_filled = df_filled[~df_filled['Sentiment'].str.contains(specific_word, case=False)]
specific_word = 'Positve'
df_filled = df_filled[~df_filled['Sentiment'].str.contains(specific_word, case=False)]
print(len(df_filled))
df_filled.head()

3007
418


,Conversation ID,Source,Text,Timestamp,ID,Sentiment
0,00036309-c615-45b0-bd95-d4669b72e66b,user,I,2023-04-17T21:15:34.492,00036309-c615-45b0-bd95-d4669b72e66b,Neutral
1,00036309-c615-45b0-bd95-d4669b72e66b,user,Yes,2023-04-17T21:16:28.619,00036309-c615-45b0-bd95-d4669b72e66b,Neutral
2,000715a4-1d49-4a54-8ee8-140808202926,user,I have a pending claim and the company fixing ...,2023-04-10T23:48:26.144,000715a4-1d49-4a54-8ee8-140808202926,Neutral
3,000c670b-4513-49d4-bf3a-daa371d4ef52,user,Hi there,2023-04-05T22:49:20.347,000c670b-4513-49d4-bf3a-daa371d4ef52,Positive
4,000c670b-4513-49d4-bf3a-daa371d4ef52,user,I hv a question do ur cover for traveller if t...,2023-04-05T22:50:25.375,000c670b-4513-49d4-bf3a-daa371d4ef52,Neutral


In [9]:
#loading data to process
data = [df_filled['Text'][0]]
for i in range (1,len(df_filled)):
    data = data + [df_filled['Text'][i]]
print(data)

['I', 'Yes', 'I have a pending claim and the company fixing my car has chosen a date and has said my excess is $1000 according to Tower. My current policy states my excess is $500. \nMy claim number is Claim numberC90154716 Hi, Neetu is looking after my claim\n\n______', 'Hi there', 'I hv a question do ur cover for traveller if they caught Covid in overseas', 'No', 'Help with my quote', 'Hi I actually wanted to cancel my policy', 'Hi Charlie bot, im trying to add my airpoints details to my account but it wont save, details are as per my card', 'i have an account', 'airpoints', 'Yes', 'Emily Robertson', '04.04.82', 'emilybroad04@gmail.com', 'Airpoints', 'Thanks Jackie', 'Thanks Jackie, have just added a screenshot', "My insurance payment was scheduled for yesterday but it didn't come out due to the public holiday, when will this payment come out??", "Would this payment usually be impacted by a public holiday? Am I still insured if it hasn't come out yet?", 'Terri McMeikan', '3/2/1992', 

In [10]:
#setting up sentiment analysis model
sentiment_pipeline = pipeline("sentiment-analysis")
specific_model1 = pipeline("text-classification", model="cardiffnlp/twitter-roberta-base-sentiment-latest")
specific_model2 = pipeline("text-classification", model="nlptown/bert-base-multilingual-uncased-sentiment")

No model was supplied, defaulted to distilbert-base-uncased-finetuned-sst-2-english and revision af0f99b (https://huggingface.co/distilbert-base-uncased-finetuned-sst-2-english).
Using a pipeline without specifying a model name and revision in production is not recommended.


Some weights of the model checkpoint at cardiffnlp/twitter-roberta-base-sentiment-latest were not used when initializing RobertaForSequenceClassification: ['roberta.pooler.dense.bias', 'roberta.pooler.dense.weight']
- This IS expected if you are initializing RobertaForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing RobertaForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


In [11]:
#processing the data
output = sentiment_pipeline(data)
output2 = specific_model1(data)
output3 = specific_model2(data)

print(output)
print(output2)
print(output3)

[{'label': 'POSITIVE', 'score': 0.9797317385673523}, {'label': 'POSITIVE', 'score': 0.9997805953025818}, {'label': 'NEGATIVE', 'score': 0.9966602325439453}, {'label': 'POSITIVE', 'score': 0.999232292175293}, {'label': 'NEGATIVE', 'score': 0.9970628619194031}, {'label': 'NEGATIVE', 'score': 0.9964459538459778}, {'label': 'NEGATIVE', 'score': 0.7868430614471436}, {'label': 'NEGATIVE', 'score': 0.998164713382721}, {'label': 'NEGATIVE', 'score': 0.9794883131980896}, {'label': 'POSITIVE', 'score': 0.995243489742279}, {'label': 'POSITIVE', 'score': 0.9915928840637207}, {'label': 'POSITIVE', 'score': 0.9997805953025818}, {'label': 'POSITIVE', 'score': 0.8187019228935242}, {'label': 'POSITIVE', 'score': 0.969683051109314}, {'label': 'NEGATIVE', 'score': 0.9477724432945251}, {'label': 'POSITIVE', 'score': 0.9915928840637207}, {'label': 'POSITIVE', 'score': 0.9998295307159424}, {'label': 'POSITIVE', 'score': 0.99969482421875}, {'label': 'NEGATIVE', 'score': 0.9984514713287354}, {'label': 'NEGATI

In [12]:
sentiment = []
score = []

for i in range (1,10):
    data = df_filled['Text'][i]
    output = specific_model1(data)
    sentiment = sentiment + [output[0]["label"]]
    score = score +[output[0]["score"]]
print(sentiment)
print(score)

['positive', 'neutral', 'neutral', 'neutral', 'neutral', 'neutral', 'neutral', 'negative', 'neutral']
[0.4553409516811371, 0.49626877903938293, 0.5172150731086731, 0.7536333799362183, 0.4406496286392212, 0.7565160393714905, 0.5097617506980896, 0.6141669750213623, 0.8165284395217896]


In [13]:
sentiment_data = []

for i in range (1,10):
  sentiment_new = df_filled["Sentiment"][i]
  sentiment_data.append(sentiment_new)

print(sentiment_data)

['Neutral', 'Neutral', 'Positive', 'Neutral', 'Neutral', 'Neutral', 'Positive', 'Positive', 'Neutral']


In [14]:
sentiment_data_upper = list((map(lambda x: x.lower(), sentiment_data)))


print(sentiment)
print(sentiment_data_upper)

total = []

for i in range (len(sentiment)):
  j = sentiment[i]
  k = sentiment_data_upper[i]
  if j == k:
    print("true")
    total.append(1)
  else:
    print("false")
    total.append(0)

print(total)


['positive', 'neutral', 'neutral', 'neutral', 'neutral', 'neutral', 'neutral', 'negative', 'neutral']
['neutral', 'neutral', 'positive', 'neutral', 'neutral', 'neutral', 'positive', 'positive', 'neutral']
false
true
false
true
true
true
false
false
true
[0, 1, 0, 1, 1, 1, 0, 0, 1]


In [17]:
count = 0
for i in range (len(total)):
  t = total[i]
  if t == 1:
    count += 1

print(count)
accuracy = (count / len(total)) * 100
print(accuracy)

5
55.55555555555556


In [ ]:
#testing purposes only on how to add to array, dont run this node
data = [df_filled['Text'][0]]
data2 = [df_filled['Text'][1]]
data3 = data + data2
data4 = ["I", "Yes"]
print(data3)
print(data4)

['I', 'Yes']
['I', 'Yes']


In [ ]:
#checking output example , dont use this
print(output[0]["label"])
print(output[0]["score"])
print(len(output))
print(output)

NEGATIVE
0.9966602325439453
1
[{'label': 'NEGATIVE', 'score': 0.9966602325439453}]


In [ ]:
#dont use this
data = df_filled['Text'][4]
print(data)

KeyError: 1

In [ ]:
# Dont use after this
#Load data
df = pd.read_excel('.\Tower_Sentiment.xlsx', 'TOWER REAL') #try replacing this with excel file ???
df.shape

(154424, 10)

In [ ]:
# Checking for missing values
df.isnull().sum()

Conversation ID                                               0
Source                                                        0
Text                                                          8
Timestamp                                                     0
Dim Milestone - Dim Milestone Milestone → Description    154424
Dim Milestone - Dim Milestone Milestone → Name                0
ID                                                            0
Sentiment                                                153320
new                                                      144693
FullSentiment                                            154255
dtype: int64

In [ ]:
df.drop(['ID', 'Timestamp', 'Dim Milestone - Dim Milestone Milestone → Description', 'Dim Milestone - Dim Milestone Milestone → Name'], axis=1)

,Conversation ID,Source,Text,Sentiment,new,FullSentiment
0,00036309-c615-45b0-bd95-d4669b72e66b,user,I,Neutral,00036309-c615-45b0-bd95-d4669b72e66b,Neutral
1,00036309-c615-45b0-bd95-d4669b72e66b,bot,Hmm.. sorry I couldn't find an answer for that!,NaN,NaN,NaN
2,00036309-c615-45b0-bd95-d4669b72e66b,bot,"To have a guided conversation instead, please ...",NaN,NaN,NaN
3,00036309-c615-45b0-bd95-d4669b72e66b,bot,Are you an existing Tower customer?,NaN,NaN,NaN
4,00036309-c615-45b0-bd95-d4669b72e66b,user,Yes,Neutral,NaN,NaN
...,...,...,...,...,...,...
154419,fff5f775-0000-4bce-bb92-1106cb083a62,bot,Are you an existing Tower customer?,NaN,NaN,NaN
154420,fffbe0c3-1d6d-46e4-be1c-dee9a5ff35c5,user,Good morning,NaN,fffbe0c3-1d6d-46e4-be1c-dee9a5ff35c5,NaN
154421,fffbe0c3-1d6d-46e4-be1c-dee9a5ff35c5,bot,We love to make things right for our customers...,NaN,NaN,NaN
154422,fffbe0c3-1d6d-46e4-be1c-dee9a5ff35c5,bot,Complaints & compliments If you're not happy w...,NaN,NaN,NaN
